In [5]:
# Ensure project root on sys.path for `import src.*`
import sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
print("sys.path bootstrapped with:", PROJECT_ROOT)


sys.path bootstrapped with: /home/nelso/Documents/IA - Detecção Falhas


In [6]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn import metrics

from tqdm import tqdm

import matplotlib.pyplot as plt
from src.data.loaders import load_gecco2018_csv

In [8]:
training_data = pd.read_csv(
    'https://www.dropbox.com/s/z32q8nks8iqkiuv/waterDataTraining.csv?dl=1',
    index_col=0)
testing_data = pd.read_csv(
    'https://www.dropbox.com/s/3ptrkyisyks2us3/waterDataTesting.csv?dl=1',
    index_col=0)

In [9]:
Xtrain = training_data.iloc[:,1:-1]
Ytrain = np.array(training_data['EVENT'])

Xtest = testing_data.iloc[:,1:-1]
Ytest = np.array(testing_data['EVENT'])

In [10]:
Xtrain = Xtrain.fillna(method='ffill')
Xtest = Xtest.fillna(method='ffill')

/tmp/ipykernel_29249/3854365444.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  Xtrain = Xtrain.fillna(method='ffill')
/tmp/ipykernel_29249/3854365444.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  Xtest = Xtest.fillna(method='ffill')


In [11]:
window_trend = 24 * 60

Xtrain_trend = Xtrain.rolling(window_trend, min_periods=1).mean()
Xtest_trend = Xtest.rolling(window_trend, min_periods=1).mean()

Xtrain = Xtrain - Xtrain_trend
Xtest = Xtest - Xtest_trend

In [12]:
val = round(Xtrain.shape[0]/2)

Xval = Xtrain[val:]
Yval = Ytrain[val:]

Xtrain = Xtrain[:val]
Ytrain = Ytrain[:val]

In [13]:
means = Xtrain.mean()
stds = Xtrain.std()

Xtrain = (Xtrain - means)/stds
Xval = (Xval - means)/stds
Xtest = (Xtest - means)/stds

In [14]:
window = 30

Xtrain_ext = pd.concat([pd.DataFrame([Xtrain.iloc[0]]*(window-1)), Xtrain])
Xval_ext = pd.concat([pd.DataFrame([Xval.iloc[0]]*(window-1)), Xval])
Xtest_ext = pd.concat([pd.DataFrame([Xtest.iloc[0]]*(window-1)), Xtest])

Xtrain_3d = np.zeros((Xtrain.shape[0], window, Xtrain.shape[1]))
Xval_3d = np.zeros((Xval.shape[0], window, Xval.shape[1]))
Xtest_3d = np.zeros((Xtest.shape[0], window, Xtest.shape[1]))

for i in tqdm(range(Xtrain.shape[0])):
    data = Xtrain_ext.iloc[i:i+window,:]
    # Xtrain_3d[i] = (data - data.mean()) / data.std()
    Xtrain_3d[i] = (data - data.mean())
    # Xtrain_3d[i] = Xtrain_ext.iloc[i:i+window,:]
for i in tqdm(range(Xval.shape[0])):
    data = Xval_ext.iloc[i:i+window,:]
    # Xval_3d[i] = (data - data.mean()) / data.std()
    Xval_3d[i] = (data - data.mean())
    # Xval_3d[i] = Xval_ext.iloc[i:i+window,:]
for i in tqdm(range(Xtest.shape[0])):
    data = Xtest_ext.iloc[i:i+window,:]
    # Xtest_3d[i] = (data - data.mean()) / data.std()
    Xtest_3d[i] = (data - data.mean())
    # Xtest_3d[i] = Xtest_ext.iloc[i:i+window,:]

  0%|          | 0/69783 [00:00<?, ?it/s]

100%|██████████| 139566/139566 [00:39<00:00, 3578.49it/s]


In [15]:
n_features = Xtrain_3d.shape[2]

Xtrain_3d_split = [Xtrain_3d[:, :, i].reshape(Xtrain_3d.shape[0], Xtrain_3d.shape[1], 1) for i in range(n_features)]
Xval_3d_split = [Xval_3d[:, :, i].reshape(Xval_3d.shape[0], Xval_3d.shape[1], 1) for i in range(n_features)]
Xtest_3d_split = [Xtest_3d[:, :, i].reshape(Xtest_3d.shape[0], Xtest_3d.shape[1], 1) for i in range(n_features)]

In [16]:
n_filters = 20
kernel_size = 5

epochs = 100
batch = 32

In [17]:
n_samples, n_window, n_features = Xtrain_3d.shape
n_filters = 13

for _ in range(30):

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=n_filters*n_features, kernel_size=kernel_size, activation='relu', input_shape=(n_window, n_features)))
    # model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Conv1D(filters=n_filters*n_features, kernel_size=kernel_size, activation='relu'))
    # model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.GlobalMaxPooling1D())

    model.add(tf.keras.layers.Dense(n_filters*n_features,activation='relu'))
    # model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(n_filters*n_features,activation='relu'))
    # model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(n_filters*n_features,activation='relu'))
    # model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

    # opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(loss="binary_crossentropy", optimizer='adam')

    indices_train_true = np.nonzero(Ytrain)[0]
    indices_train_false = np.nonzero(~Ytrain)[0]
    indices_val_true = np.nonzero(Yval)[0]
    indices_val_false = np.nonzero(~Yval)[0]

    for _ in tqdm(range(epochs)):

        indices_train = np.concatenate((
            indices_train_true,
            np.random.choice(indices_train_false,
                            len(indices_train_true),
                            replace=False)))

        indices_val = np.concatenate((
            indices_val_true,
            np.random.choice(indices_val_false,
                            len(indices_val_true),
                            replace=False)))

        model.fit(Xtrain_3d[indices_train], Ytrain[indices_train],
                validation_data=(Xval_3d[indices_val], Yval[indices_val]),
                batch_size=batch, verbose=0)

    y_pred = model.predict(Xtest_3d)
    print('F1:', np.max([metrics.f1_score(Ytest, y_pred>(th/100)) for th in range(101)]))

/home/nelso/miniconda3/envs/tastefinder/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
100%|██████████| 100/100 [00:51<00:00,  1.93it/s]


4362/4362 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step


/home/nelso/miniconda3/envs/tastefinder/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


F1: 0.5386631716906947


100%|██████████| 100/100 [00:49<00:00,  2.02it/s]


...

4362/4362 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step


/home/nelso/miniconda3/envs/tastefinder/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


F1: 0.5538264762310808


100%|██████████| 100/100 [01:02<00:00,  1.61it/s]


4362/4362 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step
F1: 0.5548158011540169


In [18]:
model.summary()

Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_58 (Conv1D)              │ (None, 26, 117)        │         5,382 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_59 (Conv1D)              │ (None, 22, 117)        │        68,562 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_29         │ (None, 117)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_116 (Dense)               │ (None, 117)            │        13,806 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_117 (Dense)               │ (None, 117)            │        13,806 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_118 (Dense)               │ (None, 117)            │        13,806 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_119 (Dense)               │ (None, 1)              │           118 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 346,442 (1.32 MB)

 Trainable params: 115,480 (451.09 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 230,962 (902.20 KB)

In [19]:
n_samples, n_window, n_features = Xtrain_3d.shape
n_filters = 20

for _ in range(30):

    visible = list()
    cnn = list()
    for i in range(n_features):
        visible.append(tf.keras.layers.Input(shape=(n_window,1)))
        cnn.append(tf.keras.layers.Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu')(visible[i]))
        # cnn[i] = tf.keras.layers.Dropout(0.1)(cnn[i])
        cnn[i] = tf.keras.layers.Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu')(cnn[i])
        # cnn[i] = tf.keras.layers.Dropout(0.1)(cnn[i])
        cnn[i] = tf.keras.layers.GlobalMaxPooling1D()(cnn[i])

    dense = tf.keras.layers.concatenate(cnn)
    dense = tf.keras.layers.Dense(n_features*n_filters,activation='relu')(dense)
    # dense = tf.keras.layers.Dropout(0.1)(dense)
    dense = tf.keras.layers.Dense(n_features*n_filters,activation='relu')(dense)
    # dense = tf.keras.layers.Dropout(0.1)(dense)
    dense = tf.keras.layers.Dense(n_features*n_filters,activation='relu')(dense)
    # dense = tf.keras.layers.Dropout(0.1)(dense)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(dense)

    model = tf.keras.Model(inputs=visible, outputs=output)

    model.compile(loss="binary_crossentropy", optimizer='adam')

    indices_train_true = np.nonzero(Ytrain)[0]
    indices_train_false = np.nonzero(~Ytrain)[0]
    indices_val_true = np.nonzero(Yval)[0]
    indices_val_false = np.nonzero(~Yval)[0]
    for _ in tqdm(range(epochs)):

        indices_train = np.concatenate((
            indices_train_true,
            np.random.choice(indices_train_false,
                            len(indices_train_true),replace=False)))

        indices_val = np.concatenate((
            indices_val_true,
            np.random.choice(indices_val_false,
                            len(indices_val_true),replace=False)))

        Xtrain_under = []
        Xval_under = []
        for f in range(n_features):
            Xtrain_under.append(Xtrain_3d_split[f][indices_train])
            Xval_under.append(Xval_3d_split[f][indices_val])

        model.fit(Xtrain_under, Ytrain[indices_train], batch_size=batch,
                validation_data = (Xval_under, Yval[indices_val]),
                verbose=0)

    y_pred = model.predict(Xtest_3d_split)
    print('F1:', np.max([metrics.f1_score(Ytest, y_pred>(th/100)) for th in range(101)]))

100%|██████████| 100/100 [01:24<00:00,  1.18it/s]


4362/4362 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step
F1: 0.6186830015313936


  6%|▌         | 6/100 [00:09<02:33,  1.63s/it]


KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "functional_239"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_291     │ (None, 30, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_292     │ (None, 30, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_293     │ (None, 30, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_294     │ (None, 30, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_295     │ (None, 30, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_296     │ (None, 30, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_297     │ (None, 30, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_298     │ (None, 30, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_299     │ (None, 30, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_582 (Conv1D) │ (None, 26, 20)    │        120 │ input_layer_291[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_584 (Conv1D) │ (None, 26, 20)    │        120 │ input_layer_292[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_586 (Conv1D) │ (None, 26, 20)    │        120 │ input_layer_293[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_588 (Conv1D) │ (None, 26, 20)    │        120 │ input_layer_294[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_590 (Conv1D) │ (None, 26, 20)    │        120 │ input_layer_295[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_592 (Conv1D) │ (None, 26, 20)    │        120 │ input_layer_296[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_594 (Conv1D) │ (None, 26, 20)    │        120 │ input_layer_297[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_596 (Conv1D) │ (None, 26, 20)    │        120 │ input_layer_298[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_598 (Conv1D) │ (None, 26, 20)    │        120 │ input_layer_299[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_583 (Conv1D) │ (None, 22, 20)    │      2,020 │ conv1d_582[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_585 (Conv1D) │ (None, 22, 20)    │      2,020 │ conv1d_584[0][0]  │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 351,545 (1.34 MB)

 Trainable params: 117,181 (457.74 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 234,364 (915.49 KB)

In [ ]:
# n_samples, n_window, n_features = Xtrain_3d.shape


# for _ in range(10):

#     visible = list()
#     shared_conv1 = tf.keras.layers.Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu')
#     shared_conv2 = tf.keras.layers.Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu')
#     cnn = list()
#     for i in range(n_features):
#         visible.append(tf.keras.layers.Input(shape=(n_window,1)))
#         cnn.append(shared_conv1(visible[i]))
#         # cnn[i] = tf.keras.layers.Dropout(0.1)(cnn[i])
#         cnn[i] = shared_conv2(cnn[i])
#         # cnn[i] = tf.keras.layers.Dropout(0.1)(cnn[i])
#         cnn[i] = tf.keras.layers.GlobalMaxPooling1D()(cnn[i])

#     dense = tf.keras.layers.concatenate(cnn)
#     dense = tf.keras.layers.Dense(n_features*n_filters,activation='relu')(dense)
#     # dense = tf.keras.layers.Dropout(0.1)(dense)
#     dense = tf.keras.layers.Dense(n_features*n_filters,activation='relu')(dense)
#     # dense = tf.keras.layers.Dropout(0.1)(dense)
#     dense = tf.keras.layers.Dense(n_features*n_filters,activation='relu')(dense)
#     # dense = tf.keras.layers.Dropout(0.1)(dense)
#     output = tf.keras.layers.Dense(1,activation='sigmoid')(dense)

#     model = tf.keras.Model(inputs=visible, outputs=output)

#     model.compile(loss="binary_crossentropy", optimizer='adam')

#     indices_train_true = np.nonzero(Ytrain)[0]
#     indices_train_false = np.nonzero(~Ytrain)[0]
#     indices_val_true = np.nonzero(Yval)[0]
#     indices_val_false = np.nonzero(~Yval)[0]
#     for _ in tqdm(range(epochs)):

#         indices_train = np.concatenate((
#             indices_train_true,
#             np.random.choice(indices_train_false,
#                             len(indices_train_true),replace=False)))

#         indices_val = np.concatenate((
#             indices_val_true,
#             np.random.choice(indices_val_false,
#                             len(indices_val_true),replace=False)))

#         Xtrain_under = []
#         Xval_under = []
#         for f in range(n_features):
#             Xtrain_under.append(Xtrain_3d_split[f][indices_train])
#             Xval_under.append(Xval_3d_split[f][indices_val])

#         model.fit(Xtrain_under, Ytrain[indices_train], batch_size=batch,
#                 validation_data = (Xval_under, Yval[indices_val]),
#                 verbose=0)

#     y_pred = model.predict(Xtest_3d_split)
#     print('F1:', np.max([metrics.f1_score(Ytest, y_pred>(th/100)) for th in range(101)]))

In [ ]:
# model.summary()

In [ ]:
from google.colab import runtime
runtime.unassign()